# Assignment 2
Bradley Thompson - CS 510 LLM Winter 2024

In [29]:
from datasets import load_dataset
import evaluate

dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "english")
perplexity = evaluate.load("perplexity", module_type="metric")

# https://stackoverflow.com/questions/75886673/how-to-compute-sentence-level-perplexity-from-hugging-face-language-models
# results = perplexity.compute(model_id=model_name, add_start_token=False, predictions=[input_text])

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 870
    })
})

In [23]:
MODELS = (
    "bigscience/bloom-560m",
    "bigscience/bloom-1b1",
    "bigscience/bloom-1b7",
    "bigscience/bloomz-560m",
    "bigscience/bloomz-1b1",
    "bigscience/bloomz-1b7",
)

ID_TO_LABEL = {
    0: "negative",
    1: "neutral",
    2: "positive",
}
LABEL_TO_ID = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

DEFAULT_PROMPT_TEMPLATE = "Of the categories 'positive', 'negative' and 'neutral', the statement '%s' is"

In [36]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Following: https://huggingface.co/docs/transformers/en/tasks/sequence_classification

model_name = MODELS[0]
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(ID_TO_LABEL), id2label=ID_TO_LABEL, label2id=LABEL_TO_ID
)

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
import torch

sample = 0
text = dataset["train"][sample]["text"]
print(text)
# prompt = DEFAULT_PROMPT_TEMPLATE % text
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits
    prediction = model.config.id2label[logits.argmax().item()]
prediction

okay i\u2019m sorry but TAYLOR SWIFT LOOKS NOTHING LIKE JACKIE O SO STOP COMPARING THE TWO. c\u2019mon America aren\u2019t you sick of her yet? (sorry) 


'positive'